In [14]:
import cv2
import numpy as np
import os
from scipy.signal import wiener


path = "images/"
output_path = "output/"  
canny_path = "canny/" 
if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(canny_path):
    os.makedirs(canny_path)
files = os.listdir(path)
files = [f for f in files if f.endswith('.jpg')]


for f in files:

    image = cv2.imread(path+f)

    h, w = image.shape[:2]

    # Resize the image to a smaller size
    scale_percent = 50  # Adjust this value as needed
    new_width = int(w * scale_percent / 100)
    new_height = int(h * scale_percent / 100)
    resized_image = cv2.resize(image, (new_width, new_height))

    # apply a gaussian blur on the image
    blurred = cv2.GaussianBlur(resized_image, (5, 5), 0)

    # Convert the image to grayscale
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # Find edges in the image using canny edge detection method
    edges = cv2.Canny(gray,100,200)

    canny_output_path = os.path.join(canny_path, f"canny_{f}")
    cv2.imwrite(canny_output_path, edges)    

    contours,hierarchy=cv2.findContours(edges,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)  #retrieve the contours as a list, with simple apprximation model
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

    # show contours on image
    cnt = resized_image.copy()
    cv2.drawContours(cnt, contours, -1, (0, 255, 0), 3)

    canny_output_path = os.path.join(canny_path, f"cnt_{f}")
    cv2.imwrite(canny_output_path, cnt)

    #the loop extracts the boundary contours of the page
    for c in contours:
        p=cv2.arcLength(c,True)
        approx=cv2.approxPolyDP(c,0.02*p,True)

        if len(approx)==4:
            target=approx
            break
    
    target = target.reshape((4,2))
    approx = np.zeros((4,2),dtype = np.float32)

    add = target.sum(1)
    approx[0] = target[np.argmin(add)]
    approx[2] = target[np.argmax(add)]

    diff = np.diff(target,axis = 1)
    approx[1] = target[np.argmin(diff)]
    approx[3] = target[np.argmax(diff)]

    pts=np.float32([[0,0],[w,0],[w,h],[0,h]]) 

    op=cv2.getPerspectiveTransform(approx,pts)
    dst=cv2.warpPerspective(resized_image,op,(w,h))

    # Save the deblurred image
    warped_output_path = os.path.join(output_path, f"warped_{f}")
    cv2.imwrite(warped_output_path, dst)   
    

    # Save or display the isolated paper
    #cv2.imshow('Edges', dst)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [20]:
import cv2
import numpy as np
import os


path = "images/"
output_path = "output/"  
if not os.path.exists(output_path):
    os.makedirs(output_path)

files = os.listdir(path)
files = [f for f in files if f.endswith('.jpg')]


# loop through all photos and read, convert to grayscale, blur, threshold, and save
for f in files:

    # Load the image
    image = cv2.imread(path+f)

    height = 700

    ratio = round(height / image.shape[0], 3)
    width = int(image.shape[1] * ratio)
    dim = (width, height)
    resized = cv2.resize(image, dim) 

    # Convert the image to grayscale
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    blurred=cv2.GaussianBlur(gray,(5,5),0)

    # Find edges in the image using canny edge detection method
    edges = cv2.Canny(blurred, 0, 50)

    contours,hierarchy=cv2.findContours(edges,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)  #retrieve the contours as a list, with simple apprximation model
    contours=sorted(contours,key=cv2.contourArea,reverse=True)


    #the loop extracts the boundary contours of the page
    for c in contours:
        p=cv2.arcLength(c,True)
        approx=cv2.approxPolyDP(c,0.02*p,True)

        if len(approx)==4:
            target=approx
            break
    
    target = target.reshape((4,2))
    approx = np.zeros((4,2),dtype = np.float32)

    add = target.sum(1)
    approx[0] = target[np.argmin(add)]
    approx[2] = target[np.argmax(add)]

    diff = np.diff(target,axis = 1)
    approx[1] = target[np.argmin(diff)]
    approx[3] = target[np.argmax(diff)]

    pts=np.float32([[0,0],[800,0],[800,800],[0,800]])

    op=cv2.getPerspectiveTransform(approx,pts)
    dst=cv2.warpPerspective(resized,op,(800,800))

    # Save the deblurred image
    warped_output_path = os.path.join(output_path, f"warped{f}")
    cv2.imwrite(warped_output_path, dst)  


